In [1]:
import sys
import pandas as pd
import numpy as np

In [3]:
# prepare commensals

In [98]:
t = pd.read_excel('../../data/Comm_424_data_def.xlsx')

In [99]:
t.tail()

,Strain,Collection,Sampling date,Location,Socio-demography,ATB consomption,Sex (M or F),Age
421,170-IKSO-DOM,Ceremi,2017.0,Paris,Volunteers,No,M,24
422,113-WIGH-DOM,Ceremi,2017.0,Paris,Volunteers,No,F,55
423,880-SLHG-DOM,Ceremi,2017.0,Paris,Volunteers,No,F,24
424,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
425,* The individuals in the BIW and PF population...,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [100]:
# drop the last two rows
t = t.drop(t.tail(2).index)

# remove 53 strains from VDG collection
t = t[t.Collection != 'VDG']

# remove outlier: LBC22a
t = t[t.Strain != 'LBC22a'].reset_index()

# rename resequenced strain C001-021-c1 for 001-021-c2
t['Strain'] = t['Strain'].str.replace('C001-021-c1', '001-021-c2')

In [101]:
del t['index']

In [102]:
# rename columns
t.rename(columns={'Sampling date':'date', 'Strain':'code_rangement', 
                  'Sex (M or F)':'sex', 'Age':'age'}, inplace=True)

In [103]:
# n comensals per collection
t.groupby(['Collection', 'date']).size()

Collection  date  
Ceremi      2017.0     20
Coliville   2010.0    246
LBC         2001.0     27
PAR         2002.0     27
ROAR        2000.0     50
dtype: int64

In [104]:
# Add column for commensal
t['commensal'] = [1 if x == x
                  else 1
                  for x in t['Collection']]
# reencode columns
t['date_after2010'] = [1 if x == 2017.0
                      else 1 if x == 2010.0
                      else 0
                      for x in t['date']]
t['age_more60'] = [0 if x == "18-50"
                    else 0 if x == "20-60"
                    else 0 if x == "18-22"
                    else 0 if x < 60
                    else 1 if x > 60
                    else np.nan
                    for x in t['age'].values]
t['female'] = [1 if x == 'F'
              else 0 if x == 'M'
              else np.nan
              for x in t['sex']]
t['location'] = [0 if x == 'Paris'
                else 1 if x == 'Brittany'
                else np.nan
                for x in t['Location'].values]
# Add column for pe_urinaire and pe_digestive
t['pe_urinaire'] = [0 if x == x
                  else 0
                  for x in t['Collection']]
t['pe_digestive'] = [0 if x == x
                  else 0
                  for x in t['Collection']]

In [105]:
# drop reencoded columns
t = t.drop(columns=['sex', 'age', 'Location', 'date'])

In [106]:
# Reencode Collection columns
t['ceremi'] = [1 if x == 'Ceremi'
              else 0
              for x in t['Collection'].values]
t['coliville'] = [1 if x == 'Coliville'
              else 0
              for x in t['Collection'].values]
t['LBC'] = [1 if x == 'Ceremi'
              else 0
              for x in t['Collection'].values]
t['PAR'] = [1 if x == 'PAR'
              else 0
              for x in t['Collection'].values]
t['ROAR'] = [1 if x == 'ROAR'
              else 0
              for x in t['Collection'].values]
# Add columns for Septicoli and Colibafi collections of BSI isolates
t['septicoli'] = [0 if x == x
              else 0
              for x in t['Collection'].values]
t['colibafi'] = [0 if x == x
              else 0
              for x in t['Collection'].values]

In [107]:
# drop reencoded collection column and other columns
# ATB consomtion: 343 No, 27 Unlikely
# Socio-demography: we don't have the one from the BSI strains
# location: we don't have it from the BSI strains
t = t.drop(columns=['Socio-demography', 'ATB consomption', 'Collection', 'location'])

In [108]:
t.head()

,code_rangement,commensal,date_after2010,age_more60,female,pe_urinaire,pe_digestive,ceremi,coliville,LBC,PAR,ROAR,septicoli,colibafi
0,C001-001-P1,1,1,0.0,0.0,0,0,0,1,0,0,0,0,0
1,C001-001-P2,1,1,NaN,0.0,0,0,0,1,0,0,0,0,0
2,C001-002,1,1,0.0,1.0,0,0,0,1,0,0,0,0,0
3,C001-003-cb,1,1,1.0,0.0,0,0,0,1,0,0,0,0,0
4,C001-004-ccl,1,1,0.0,0.0,0,0,0,1,0,0,0,0,0


In [110]:
# save commensal reencoded dataset
t.to_csv('../../data/commensal_reencoded.csv', index=False, sep='\t')

In [2]:
p = pd.read_csv('../../data/phenotype.tsv', sep='\t')

In [4]:
pp = p[p.commensal == 1]

In [7]:
pp = pp[['code_rangement', 'commensal', 'phylogroup']]

In [10]:
pp.to_csv('../../data/commensal_temporal.csv', index=False, sep='\t')

_________

In [4]:
# prepare full dataset

In [ ]:
c = pd.read_csv('../../data/clinical_full.tsv', sep='\t')
r = pd.read_csv('../../data/commensal_reencoded.csv', sep='\t')

In [ ]:
c = c[['code_rangement', 'age', 'septicoli', 'female', 'pe_digestive', 'pe_urinaire']]

In [ ]:
# Add commensal column
c['commensal'] = [0 if x == 1
                 else 0
                 for x in c['code_rangement']]
# Add colibafi column
c['colibafi'] = [1 if x == 0
                else 0
                for x in c['septicoli']]
# Reencode columns
c['age_more60'] = [0 if x < 60
                    else 1 if x > 60
                    else np.nan
                    for x in c['age'].values]
# Add date column
# colibafi = 2005
# septicoli = 2016-2017
c['date_after2010'] = [1 if x == 1
                      else 0 if x == 0
                      else np.nan
                      for x in c['septicoli']]

In [ ]:
# drop reencoded columns
c = c.drop(columns=['age'])

In [ ]:
# add other collections column
c['ceremi'] = [0 if x == x
               else 0
               for x in c['septicoli'].values]
c['coliville'] = [0 if x == x
                  else 0
                  for x in c['septicoli'].values]
c['LBC'] = [0 if x == x
            else 0
            for x in c['septicoli'].values]
c['PAR'] = [0 if x == x
            else 0
            for x in c['septicoli'].values]
c['ROAR'] = [0 if x == x
             else 0
             for x in c['septicoli'].values]

In [ ]:
# reorder as commensal dataset
c = c[['code_rangement', 'commensal', 'date_after2010', 'age_more60', 'female',
       'pe_urinaire', 'pe_digestive', 'ceremi', 'coliville', 'LBC', 'PAR', 'ROAR',
       'septicoli', 'colibafi']]

In [ ]:
r

In [ ]:
# merge both datasets into full set
f = pd.concat([c,r], axis=0, ignore_index=True)

In [ ]:
f.groupby('commensal').size()

In [ ]:
f

In [ ]:
# save clinical reencoded and full dataset
c.to_csv('../../data/clinical_reencoded.csv', index=False, sep='\t')
f.to_csv('../../data/full_reencoded.csv', index=False, sep='\t')

_________

In [ ]:
# prepare subsets with portal of entry as urinary tract or digestive tract

In [ ]:
# load imputed data
i = pd.read_csv('../../data/full_imputed.csv', sep='\t')

In [ ]:
i.head()

In [ ]:
# get commensals
c = i[i.commensal == 1]
# get pe_urinaire and pe_digestive
peu = i[i.pe_urinaire == 1]
ped = i[i.pe_digestive == 1]

In [ ]:
# get full peu_commensals and ped_commensals datasets
cu = pd.concat([c,peu], axis=0, ignore_index=True)
cd = pd.concat([c,ped], axis=0, ignore_index=True)

In [ ]:
# save imputed dataframes of pe_digestive and pe_urinaire
cu.to_csv('../../data/pe_urinaire_imputed.csv', index=False, sep='\t')
cd.to_csv('../../data/pe_digestive_imputed.csv', index=False, sep='\t')